# 2D mesh tallies

### This example makes a simple geometry from a few different materials and plots a 2D regular mesh tally of the results.

### A mesh tally allows a visual inspection of the neutron behaviour spatially throughout the geometry.

### The geometry is subdivided into many rectangles and the neutron behaviour is recorded (tallied) by the simulation in each of the small rectangles.

### This can be formed into a 2D slice of the neutron reactions through the model

In [ ]:
import openmc
import matplotlib.pyplot as plt

# MATERIALS

# creates two materials, one is a neutron multiplier (lead) and the other an 
mats = openmc.Materials()
 
breeder_material = openmc.Material(name="breeder") 
breeder_material.add_element('Li', 1, percent_type='ao')
breeder_material.set_density('g/cm3', 2.0)

multiplier_material = openmc.Material(name="multiplier") 
multiplier_material.add_element('Pb', 1, percent_type='ao')
multiplier_material.set_density('g/cm3', 11.0)

mats = [breeder_material, multiplier_material]



# GEOMETRY

# surfaces
sph1 = openmc.Sphere(r=50)
sph2 = openmc.Sphere(r=90, boundary_type='vacuum')
plane1 = openmc.XPlane(20)

# cells
breeder_cell = openmc.Cell(region=+sph1 & -sph2 & -plane1)
breeder_cell.fill = breeder_material

multiplier_cell = openmc.Cell(region=+sph1 & -sph2 & +plane1)
multiplier_cell.fill = multiplier_material

inner_vacuum_cell = openmc.Cell(region=-sph1)

universe = openmc.Universe(cells=[inner_vacuum_cell, breeder_cell, multiplier_cell])

geom = openmc.Geometry(universe)


# SETTINGS

# Instantiate a Settings object
sett = openmc.Settings()
sett.batches = 100
sett.inactive = 0
sett.particles = 50
sett.particle = "neutron"
sett.run_mode = 'fixed source'

# creates a 14MeV point source
source = openmc.Source()
source.space = openmc.stats.Point((0, 0, 0))
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([14e6], [1])
sett.source = source


# Create mesh which will be used for tally
mesh = openmc.RegularMesh()
mesh_height = 100
mesh_width = mesh_height
mesh.dimension = [mesh_width, mesh_height]
mesh.lower_left = [-200, -200]
mesh.upper_right = [200, 200]


tallies = openmc.Tallies()
# Create mesh filter for tally
mesh_filter = openmc.MeshFilter(mesh)
mesh_tally = openmc.Tally(name='tallies_on_mesh')
mesh_tally.filters = [mesh_filter]
mesh_tally.scores = ['flux', 'absorption', '(n,2n)']  # change flux to absorption
tallies.append(mesh_tally)

# combines the geometry, materials, settings and tallies to create a neutronics model
model = openmc.model.Model(geom, mats, sett, tallies)

plt.show(universe.plot(width=(180, 180), basis='xz'))

### This performs the simulation and loads up the results for inspection

In [ ]:
# deletes old files
!rm summary.h5
!rm statepoint.*.h5

# runs the simulation
output_filename = model.run()

# open the results file
results = openmc.StatePoint(output_filename)

### This filters the results to show the neutron flux recorded on the mesh tally

In [ ]:
# access the flux tally
my_tally = results.get_tally(scores=['flux'])
my_slice = my_tally.get_slice(scores=['flux'])
my_slice.mean.shape = (mesh_width, mesh_height)

fig = plt.subplot()

plt.show(fig.imshow(my_slice.mean))

# notice that neutrons are produced and emitted isotropically from a point source.
# There is a slight increase in flux within the neutron multiplier.

### This filters the results to show the neutron absorption recorded on the mesh tally

In [ ]:
# access the absorption tally
my_tally = results.get_tally(scores=['absorption'])
my_slice = my_tally.get_slice(scores=['absorption'])
my_slice.mean.shape = (mesh_width, mesh_height)

fig = plt.subplot()

plt.show(fig.imshow(my_slice.mean))

# notice that neutrons are being absorpted on the left hand side of the model

### This filters the results to show the neutron multiplication recorded on the mesh tally

In [ ]:
# access the neutron multiplication tally
my_tally = results.get_tally(scores=['(n,2n)'])
my_slice = my_tally.get_slice(scores=['(n,2n)'])
my_slice.mean.shape = (mesh_width, mesh_height)

fig = plt.subplot()

plt.show(fig.imshow(my_slice.mean))

# notice that neutrons are being muliplied on the right hand side of the model